## Integrantes: Fausto Quizhpi y Jean Pierre Zambrano

# 9.5 Recurrent Neural Network Implementation from Scratch


We are now ready to implement an RNN from scratch.
In particular, we will train this RNN to function
as a character-level language model
(see <u> Section 9.4</u>)
and train it on a corpus consisting of 
the entire text of H. G. Wells' *The Time Machine*,
following the data processing steps 
outlined in <u> Section 9.2</u>.
We start by loading the dataset.

In [1]:
use strict;
use warnings;
use Data::Dump qw(dump);
use AI::MXNet qw(mx);
use d2l;
IPerl->load_plugin('Chart::Plotly');
use List::Util qw(sum);
use jjap::numperl;
use constant np => 'numperl';
use experimental 'smartmatch';

## 9.5.1. RNN Model

We begin by defining a class 
to implement the RNN model
(<u>Section 9.4.2</u>).
Note that the number of hidden units `num_hiddens` 
is a tunable hyperparameter.

In [2]:
package RNNScratch{
    use base qw(d2l::Module); #@save
    #The RNN model implemented from scratch.
    sub new{
        my ($class, %args) = (shift, num_inputs => undef, 
                                    num_hiddens => undef, 
                                    sigma => 0.01, @_);

        my $self = $class->SUPER::new();
        $self->save_hyperparameters(%args);

        $self->{W_xh} = mx->nd->random->randn($args{num_inputs}, $args{num_hiddens}) * $args{sigma};
        $self->{W_hh} = mx->nd->random->randn($args{num_hiddens}, $args{num_hiddens}) * $args{sigma};
        $self->{b_h} = mx->nd->zeros([$args{num_hiddens}]);
        return bless($self, $class);
    }
    1;    
}

1

The `forward` method below defines how to compute 
the output and hidden state at any time step,
given the current input and the state of the model
at the previous time step.
Note that the RNN model loops through 
the outermost dimension of `inputs`,
updating the hidden state 
one time step at a time.
The model here uses a $\tanh$ activation function (<u> Section 5.1.2.3</u>).

In [3]:
my $forward = sub{
    my ($self, $inputs, $state) = @_;
                            
    if (!defined $state){
        # Initial state with shape: (batch_size, num_hiddens)
        $state = mx->nd->zeros([$inputs->shape->[1], $self->{num_hiddens}], ctx => $inputs->context );
    }else{
        ($state, undef) = @$state;        
    }
    my $outputs;
    my $first_dot;
    my $second_dot;
    my $total;
    foreach my $X (@$inputs){ # Shape of inputs: (num_steps, batch_size, num_inputs)
        $first_dot = mx->nd->dot($X, $self->{W_xh});
        $second_dot = mx->nd->dot($state, $self->{W_hh});
        $total = $first_dot + $second_dot + $self->{b_h};
        $state = mx->nd->tanh($total);
        push @$outputs, $state;

    }
    
    return $outputs, $state;
    
};
d2l->add_to_class('RNNScratch', 'forward', $forward);

*RNNScratch::forward

We can feed a minibatch of input sequences into an RNN model as follows.

In [4]:
my ($batch_size, $num_inputs, $num_hiddens, $num_steps) = (2, 16, 32, 100);
my $rnn = RNNScratch->new(num_inputs => $num_inputs, 
                            num_hiddens => $num_hiddens);
my $X = mx->nd->ones([$num_steps, $batch_size, $num_inputs]);
my ($outputs, $state) = $rnn->forward($X);


ARRAY(0xaba0f10)<AI::MXNet::NDArray 2x32 @cpu(0)>

Let's check whether the RNN model
produces results of the correct shapes
to ensure that the dimensionality 
of the hidden state remains unchanged.


In [5]:
sub check_len{ #@save
    my ($a, $n) = @_;
    #Check the length of a list.
    if (ref ($a) eq 'AI::MXNet::NDArray'){
        if($a->len != $n){
        print STDERR " list's length ", $a->len, " != expected length ", $n;
        }
    }
    if (ref($a) eq 'ARRAY'){
        if(scalar @$a != $n){
            print STDERR " list's length ", scalar @$a, " != expected length ", $n;
        }
    }

}

sub check_shape{ #@save
    my ($a, $shape) = @_;
    #Check the shape of a tensor.
    print STDERR "tensor's shape", dump ($a->shape), " != expected shape ", dump ($shape) unless (@{$a->shape} ~~ @{$shape});


}

check_len($outputs, $num_steps);
check_shape($outputs->[0], [$batch_size, $num_hiddens]);
check_shape($state, [$batch_size, $num_hiddens]);

1

## 9.5.2. RNN-Based Language Model

The following `RNNLMScratch` class defines 
an RNN-based language model,
where we pass in our RNN 
via the `rnn` argument
of the `__init__` method.
When training language models, 
the inputs and outputs are 
from the same vocabulary. 
Hence, they have the same dimension,
which is equal to the vocabulary size.
Note that we use perplexity to evaluate the model. 
As discussed in <u>Section 9.3.2</u>, this ensures 
that sequences of different length are comparable.

In [6]:
package RNNLMScratch{
    use base qw(d2l::Classifier); #@save
    use Data::Dump qw(dump);    
    use jjap::numperl;
    use constant np => 'numperl';
    #The RNN-based language model implemented from scratch.
sub new{
        my ($class, %args) = (shift, rnn => undef, 
                                    vocab_size => undef, 
                                    lr => 0.01, @_);
        my $self = $class->SUPER::new();
        $self->save_hyperparameters(%args);
        $self->init_params();
        return bless($self, $class);
    }
    sub init_params{
        my $self = shift;
        $self->{W_hq} = mx->nd->random->randn(
            $self->{rnn}->{num_hiddens}, $self->{vocab_size}) * $self->{rnn}->{sigma};
        $self->{b_q} = mx->nd->zeros([$self->{vocab_size}]);
        for my $param (@{$self->get_scratch_params()}){
            $param->attach_grad();
        }       
    }
    sub training_step{
        my ($self, $batch) = @_; #batch es un array
        my $l = $self->loss($self->forward(@$batch[0 .. scalar(@$batch) - 2]), @$batch[1]);
        $self->plot('ppl', mx->nd->array(np->exp($l->asarray)->asarray), train => 1);
        return $l;
    }

    sub validation_step{
        my ($self, $batch) = @_;
        my $l = $self->loss($self->forward(@$batch[0 .. scalar(@$batch) - 2]), @$batch[1]);
        $self->plot('ppl', mx->nd->array(np->exp($l->asarray)->asarray), train => 0);
    }
        
    
    1;    
}

1

### 9.5.2.1. One-Hot Encoding

Recall that each token is represented 
by a numerical index indicating the
position in the vocabulary of the 
corresponding word/character/word piece.
You might be tempted to build a neural network
with a single input node (at each time step),
where the index could be fed in as a scalar value.
This works when we are dealing with numerical inputs 
like price or temperature, where any two values
sufficiently close together
should be treated similarly.
But this does not quite make sense. 
The $45^{\textrm{th}}$ and $46^{\textrm{th}}$ words 
in our vocabulary happen to be "their" and "said",
whose meanings are not remotelproblem`).
A one-hot encoding is a vector whose length
is given by the size of the vocabulary $N$,
where all entries are set to $0$,
except for the entry corresponding 
to our token, which is set to $1$.
For example, if the vocabulary had five elements,
then the one-hot vectors corresponding 
to indices 0 and 2 would be the following.y similar.

When dealing with such categorical data,
the most common strategy is to represent
each item by a *one-hot encoding*
(recall from <u> Section 4.1.1</u>). A one-hot encoding is a vector whose length is given by the size of the vocabulary  N , where all entries are set to  0 , except for the entry corresponding to our token, which is set to  1 . For example, if the vocabulary had five elements, then the one-hot vectors corresponding to indices 0 and 2 would be the following.

In [7]:
mx->nd->one_hot(mx->nd->array([0,2]),5)->aspdl;


[
 [1 0 0 0 0]
 [0 0 1 0 0]
]


The minibatches that we sample at each iteration
will take the shape (batch size, number of time steps).
Once representing each input as a one-hot vector,
we can think of each minibatch as a three-dimensional tensor, 
where the length along the third axis 
is given by the vocabulary size (`len(vocab)`).
We often transpose the input so that we will obtain an output 
of shape (number of time steps, batch size, vocabulary size).
This will allow us to loop more conveniently through the outermost dimension
for updating hidden states of a minibatch,
time step by time step
(e.g., in the above `forward` method).


In [8]:
my $one_hot = sub {
my ($self, $X) = @_;
    # Output shape: (num_steps, batch_size, vocab_size)
    return mx->nd->one_hot($X->transpose, $self->{vocab_size});
};
d2l->add_to_class('RNNLMScratch', 'one_hot', $one_hot);


*RNNLMScratch::one_hot

### 9.5.2.2. Transforming RNN Outputs

The language model uses a fully connected output layer
to transform RNN outputs into token predictions at each time step.


In [9]:
my $output_layer = sub{
    my ($self, $rnn_outputs) = @_;
    my @outputs;
    foreach my $H (@$rnn_outputs){
        push (@outputs, (mx->nd->dot($H, $self->{W_hq}) + $self->{b_q}));
    }
    return mx->nd->stack(@outputs, axis => 1);
};
d2l->add_to_class('RNNLMScratch', 'output_layer', $output_layer); #@save

*RNNLMScratch::output_layer

In [10]:
my $forward = sub{
    my ($self, $X, $state) = @_;
    my $embs = $self->one_hot($X);
    (my $rnn_outputs, $state) = $self->{rnn}->forward($embs, $state);
    return $self->output_layer($rnn_outputs);
    
};
d2l->add_to_class('RNNLMScratch', 'forward', $forward); #@save


*RNNLMScratch::forward

Let's check whether the forward computation
produces outputs with the correct shape.

In [11]:
my $model = RNNLMScratch->new(rnn => $rnn, vocab_size => $num_inputs);
my $outputs = $model->forward(mx->nd->ones([$batch_size, $num_steps], dtype => 'int64')); 
check_shape($outputs, [$batch_size, $num_steps, $num_inputs]);

1

## 9.5.3 Gradient Clipping


While you are already used to thinking of neural networks
as "deep" in the sense that many layers
separate the input and output 
even within a single time step,
the length of the sequence introduces
a new notion of depth.
In addition to the passing through the network
in the input-to-output direction,
inputs at the first time step
must pass through a chain of $T$ layers
along the time steps in order 
to influence the output of the model
at the final time step.
Taking the backwards view, in each iteration,
we backpropagate gradients through time,
resulting in a chain of matrix-products 
of length  $\mathcal{O}(T)$.
As mentioned in <u>Section 5.4 </u>, 
this can result in numerical instability, 
causing the gradients either to explode or vanish,
depending on the properties of the weight matrices. 

Dealing with vanishing and exploding gradients 
is a fundamental problem when designing RNNs
and has inspired some of the biggest advances
in modern neural network architectures.
In the next chapter, we will talk about
specialized architectures that were designed
in hopes of mitigating the vanishing gradient problem.
However, even modern RNNs often suffer
from exploding gradients.
One inelegant but ubiquitous solution
is to simply clip the gradients 
forcing the resulting "clipped" gradients
to take smaller values. 


Generally speaking, when optimizing some objective
by gradient descent, we iteratively update
the parameter of interest, say a vector $\mathbf{x}$,
but pushing it in the direction of the 
negative gradient $\mathbf{g}$
(in stochastic gradient descent, 
we calculate this gradient
on a randomly sampled minibatch).
For example, with learning rate $\eta > 0$,
each update takes the form 
$\mathbf{x} \gets \mathbf{x} - \eta \mathbf{g}$.
Let's further assume that the objective function $f$
is sufficiently smooth. 
Formally, we say that the objective 
is *Lipschitz continuous* with constant $L$,
meaning that for any $\mathbf{x}$ and $\mathbf{y}$, we have

$$|f(\mathbf{x}) - f(\mathbf{y})| \leq L \|\mathbf{x} - \mathbf{y}\|.$$

As you can see, when we update the parameter vector by subtracting $\eta \mathbf{g}$,
the change in the value of the objective
depends on the learning rate,
the norm of the gradient and $L$ as follows:

$$|f(\mathbf{x}) - f(\mathbf{x} - \eta\mathbf{g})| \leq L \eta\|\mathbf{g}\|.$$

In other words, the objective cannot
change by more than $L \eta \|\mathbf{g}\|$. 
Having a small value for this upper bound 
might be viewed as good or bad.
On the downside, we are limiting the speed
at which we can reduce the value of the objective.
On the bright side, this limits by just how much
we can go wrong in any one gradient step.


When we say that gradients explode, 
we mean that $\|\mathbf{g}\|$ 
becomes excessively large.
In this worst case, we might do so much
damage in a single gradient step that we
could undo all of the progress made over
the course of thousands of training iterations.
When gradients can be so large,
neural network training often diverges,
failing to reduce the value of the objective.
At other times, training eventually converges
but is unstable owing to massive spikes in the loss.


One way to limit the size of $L \eta \|\mathbf{g}\|$ 
is to shrink the learning rate $\eta$ to tiny values.
This has the advantage that we do not bias the updates.
But what if we only *rarely* get large gradients?
This drastic move slows down our progress at all steps,
just to deal with the rare exploding gradient events.
A popular alternative is to adopt a *gradient clipping* heuristic
projecting the gradients $\mathbf{g}$ onto a ball 
of some given radius $\theta$ as follows:

(**$$\mathbf{g} \leftarrow \min\left(1, \frac{\theta}{\|\mathbf{g}\|}\right) \mathbf{g}.$$**)

This ensures that the gradient norm never exceeds $\theta$ 
and that the updated gradient is entirely aligned 
with the original direction of $\mathbf{g}$.
It also has the desirable side-effect 
of limiting the influence any given minibatch 
(and within it any given sample) 
can exert on the parameter vector. 
This bestows a certain degree of robustness to the model. 
To be clear, it is a hack. 
Gradient clipping means that we are not always
following the true gradient and it is hard 
to reason analytically about the possible side effects.
However, it is a very useful hack,
and is widely adopted in RNN implementations
in most deep learning frameworks.


Below we define a method to clip gradients,
which is invoked by the `fit_epoch` method of
the `d2l.Trainer` class (see <u> Section 3.4 </u>).
Note that when computing the gradient norm,
we are concatenating all model parameters,
treating them as a single giant parameter vector.


In [12]:
my $clip_gradients = sub{
my ($self, $grad_clip_val, $model) = @_;
    my $params = $model->parameters();    
    if (ref $params ne 'ARRAY') {
        $params = map { $_->data() } values %{$params};
    }

    my $norm = 0;
    foreach my $param (@$params) {
        $norm += ($param->grad() ** 2)->sum();
    }
    $norm = sqrt($norm);

    if ($norm > $grad_clip_val) {
        foreach my $param (@$params) {
            $param->grad()->[':'] *= $grad_clip_val / $norm;
        }
    }
};
d2l->add_to_class('d2l::Trainer', 'clip_gradients', $clip_gradients);


*d2l::Trainer::clip_gradients

Warning: Subroutine d2l::Trainer::clip_gradients redefined at /usr/local/lib/perl5/site_perl/5.32.1/x86_64-linux/d2l.pm line 4450.


## 9.5.4. Training

Using *The Time Machine* dataset (`data`),
we train a character-level language model (`model`)
based on the RNN (`rnn`) implemented from scratch.
Note that we first calculate the gradients,
then clip them, and finally 
update the model parameters
using the clipped gradients.

In [13]:
my $data = d2l::TimeMachine->new(batch_size => 1024, num_steps => 32);
my $rnn = RNNScratch->new(num_inputs => $data->{vocab}->len, num_hiddens => 32);
my $model = RNNLMScratch->new(rnn => $rnn, vocab_size => $data->{vocab}->len, lr => 1);
my $trainer = d2l::Trainer->new(max_epochs => 100, gradient_clip_val => 1, num_gpus => 1);
$trainer->fit($model, $data);


Error: Attribute (shuffle) does not pass the type constraint because: Validation failed for 'Bool' with value ARRAY(0xc28c8b8) at /usr/local/lib/perl5/site_perl/5.32.1/x86_64-linux/Mouse/Util.pm line 395.
	Mouse::Util::throw_error(Mouse::Meta::Attribute=HASH(0x7dd1c80), "Attribute (shuffle) does not pass the type constraint because"..., "data", ARRAY(0xc28c8b8), "depth", -1) called at /usr/local/lib/perl5/site_perl/5.32.1/AI/MXNet/Gluon/Data/Loader.pm line 67
	AI::MXNet::Gluon::Data::Loader::DataLoader::__ANON__("AI::MXNet::Gluon::Data", AI::MXNet::Gluon::Data::ArrayDataset=HASH(0xc28cc78), "batch_size", 1024, "shuffle", ARRAY(0xc28c8b8), "last_batch", "keep") called at /usr/local/lib/perl5/site_perl/5.32.1/x86_64-linux/d2l.pm line 6787
	d2l::DataModule::get_tensorloader(undef, undef) called at /usr/local/lib/perl5/site_perl/5.32.1/x86_64-linux/d2l.pm line 7480
	d2l::TimeMachine::get_dataloader(d2l::TimeMachine=HASH(0xc242268), "train", 1) called at /usr/local/lib/perl5/site_perl/5.32.

## 9.5.5. Decoding

Once a language model has been learned,
we can use it not only to predict the next token
but to continue predicting each subsequent one,
treating the previously predicted token as though
it were the next in the input. 
Sometimes we will just want to generate text
as though we were starting at the beginning 
of a document. 
However, it is often useful to condition
the language model on a user-supplied prefix.
For example, if we were developing an
autocomplete feature for a search engine
or to assist users in writing emails,
we would want to feed in what they 
had written so far (the prefix), 
and then generate a likely continuation.


The following `predict` method
generates a continuation, one character at a time,
after ingesting a user-provided `prefix`.
When looping through the characters in `prefix`,
we keep passing the hidden state
to the next time step 
but do not generate any output.
This is called the *warm-up* period.
After ingesting the prefix, we are now
ready to begin emitting the subsequent characters,
each of which will be fed back into the model 
as the input at the next time step.


In [14]:
# Definir la subrutina anónima $predict para generar texto a partir de un prefijo
my $predict = sub {
    my ($self, $prefix, $num_preds, $vocab, $device) = @_;
    
    # Inicializar el estado y la matriz de salidas con el primer token del prefijo
    my ($state, @outputs) = (undef, $vocab->{$prefix->[0]});
    
    # Iterar para generar el texto deseado
    for my $i (0..(scalar(@$prefix) + $num_preds - 1)) {
    
        # Crear un tensor con el último token de la matriz de salidas
        my $X = mx->nd->array([[$outputs[-1]]], ctx => $device);
       
        # Obtener la representación one-hot del tensor X
        my $embs = $self->one_hot($X);
        
        # Realizar un paso hacia adelante en la red RNN y obtener las salidas y el nuevo estado
        my ($rnn_outputs, $state) = $self->{rnn}->forward($embs, $state);

        # Si estamos en el período de calentamiento (antes de generar el texto real)
        if ($i < scalar(@$prefix) - 1) {  
        
            # Agregar el siguiente token del prefijo a la matriz de salidas
            push @outputs, $vocab->{$prefix->[$i + 1]};
            
        } else {  # Generar texto real
        
            # Obtener la capa de salida y encontrar el índice del token con mayor probabilidad
            my $Y = $self->output_layer($rnn_outputs);
            my $index = int($Y->argmax(axis => 2)->aspdl->at(0));
            
            # Agregar el índice como próximo token a la matriz de salidas
            push @outputs, $index;
}

    }
    # Convertir los índices de tokens en sus representaciones de texto correspondientes
    my @tokens = map { $vocab->{idx_to_token}->[$_] // '<unk>' } @outputs;
    
    # Unir los tokens generados para formar el texto completo
    return join('', @tokens);
};
# Agregar la subrutina $predict a la clase RNNLMScratch en el módulo d2l
d2l->add_to_class('RNNLMScratch', 'predict', $predict);

*RNNLMScratch::predict

In the following, we specify the prefix 
and have it generate 20 additional characters.

In [15]:
 $model->predict(['it', 'has'], 20, $data->{vocab}, d2l::try_gpu());


  cqqqqqqqqqqqqqqqqqqqq

Warning: Use of uninitialized value $_ in array element at reply input line 38.

Use of uninitialized value $_ in array element at reply input line 38.


While implementing the above RNN model from scratch is instructive, it is not convenient.
In the next section, we will see how to leverage deep learning frameworks to whip up RNNs
using standard architectures, and to reap performance gains 
by relying on highly optimized library functions.


## Summary

We can train RNN-based language models to generate text following the user-provided text prefix. 
A simple RNN language model consists of input encoding, RNN modeling, and output generation.
During training, gradient clipping can mitigate the problem of exploding gradients but does not address the problem of vanishing gradients. In the experiment, we implemented a simple RNN language model and trained it with gradient clipping on sequences of text, tokenized at the character level. By conditioning on a prefix, we can use a language model to generate likely continuations, which proves useful in many applications, e.g., autocomplete features.


## Exercises

1. Does the implemented language model predict the next token based on all the past tokens up to the very first token in *The Time Machine*? 
1. Which hyperparameter controls the length of history used for prediction?
1. Show that one-hot encoding is equivalent to picking a different embedding for each object.
1. Adjust the hyperparameters (e.g., number of epochs, number of hidden units, number of time steps in a minibatch, and learning rate) to improve the perplexity. How low can you go while sticking with this simple architecture?
1. Replace one-hot encoding with learnable embeddings. Does this lead to better performance?
1. Conduct an experiment to determine how well this language model 
   trained on *The Time Machine* works on other books by H. G. Wells,
   e.g., *The War of the Worlds*.
1. Conduct another experiment to evaluate the perplexity of this model
   on books written by other authors. 
1. Modify the prediction method so as to use sampling 
   rather than picking the most likely next character.
    * What happens?
    * Bias the model towards more likely outputs, e.g., 
    by sampling from $q(x_t \mid x_{t-1}, \ldots, x_1) \propto P(x_t \mid x_{t-1}, \ldots, x_1)^\alpha$ for $\alpha > 1$.
1. Run the code in this section without clipping the gradient. What happens?
1. Replace the activation function used in this section with ReLU 
   and repeat the experiments in this section. Do we still need gradient clipping? Why?
